In [1]:
import pandas as pd
import requests
import json
import dml
import prov.model
import datetime
import uuid
import csv 
from io import StringIO
import json
import pymongo
import math

In [2]:
# define relational models
def union(R, S):
    return R + S

def difference(R, S):
    return [t for t in R if t not in S]

def intersect(R, S):
    return [t for t in R if t in S]

def project(R, p):
    return [p(t) for t in R]

def select(R, s):
    return [t for t in R if s(t)]
 
def product(R, S):
    return [(t,u) for t in R for u in S]

def aggregate(R, f):
    keys = {r[0] for r in R}
    return [(key, f([v for (k,v) in R if k == key])) for key in keys]

In [3]:
contributor = 'mmao95_Dongyihe_weijiang_zhukk'
reads = []
writes = [contributor + '.public_libraries']

In [4]:
startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

True

In [5]:
url = 'http://bostonopendata-boston.opendata.arcgis.com/datasets/cb00f9248aa6404ab741071ca3806c0e_6.csv'
doc = requests.get(url).text
data_file = StringIO(doc)
reader = csv.reader(data_file)
Public_Libraries_list = []
for row in reader:
    Public_Libraries_list += [row]

del Public_Libraries_list[0]

BSDZXYL = [(BRANCH, ST_ADDRESS, DISTRICT, ZIPCODE, X, Y, LIBRARIES) for [X, Y, OBJECTID_1, OBJECTID, LIBRARIES, DISTRICT, ST_ADDRESS, BRANCH, ZIPCODE] in Public_Libraries_list]

columnName = ['Branch Name', 'Address', 'City', 'Zipcode', 'Latitude', 'Longitude', 'Numbers']
df = pd.DataFrame(columns = columnName, data = BSDZXYL)
data = json.loads(df.to_json(orient="records"))

In [6]:
repo.dropCollection('public_libraries')
repo.createCollection('public_libraries')
repo[writes[0]].insert_many(data)

In [7]:
repo[writes[0]].metadata({'complete': True})
print(repo[writes[0]].metadata())
[record for record in repo[writes[0]].find()]

{'_id': ObjectId('5c802ea81fcf2f427c584490'), 'complete': True}


[{'_id': ObjectId('5c802ea81fcf2f427c584476'),
  'Branch Name': 'West End',
  'Address': '151 Cambridge St',
  'City': ' ',
  'Zipcode': '02114',
  'Latitude': '-71.065006409545546',
  'Longitude': '42.361446684455274',
  'Numbers': '5'},
 {'_id': ObjectId('5c802ea81fcf2f427c584477'),
  'Branch Name': 'Codman Square',
  'Address': '690 Washington St',
  'City': ' ',
  'Zipcode': '02124',
  'Latitude': '-71.070968053188679',
  'Longitude': '42.287519203629671',
  'Numbers': '11'},
 {'_id': ObjectId('5c802ea81fcf2f427c584478'),
  'Branch Name': 'Hyde Park',
  'Address': '35 Harvard Av',
  'City': 'HYDE PARK',
  'Zipcode': '02136',
  'Latitude': '-71.122135805346076',
  'Longitude': '42.257104012218086',
  'Numbers': '17'},
 {'_id': ObjectId('5c802ea81fcf2f427c584479'),
  'Branch Name': 'Connolly',
  'Address': '433 Centre St',
  'City': 'JAMAICA PLAIN',
  'Zipcode': '02130',
  'Latitude': '-71.111098666378751',
  'Longitude': '42.320645768833877',
  'Numbers': '18'},
 {'_id': ObjectId('5

In [8]:
repo.logout()
endTime = datetime.datetime.now()
{"start":startTime, "end":endTime}

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Database.logout() is deprecated
  """Entry point for launching an IPython kernel.


{'start': datetime.datetime(2019, 3, 6, 15, 33, 40, 858311),
 'end': datetime.datetime(2019, 3, 6, 15, 33, 48, 121165)}

In [9]:
doc = prov.model.ProvDocument()
startTime = None
endTime = None
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

doc.add_namespace('alg', 'http://datamechanics.io/algorithm/') # The scripts are in <folder>#<filename> format.
doc.add_namespace('dat', 'http://datamechanics.io/data/') # The data sets are in <user>#<collection> format.
doc.add_namespace('ont', 'http://datamechanics.io/ontology#') # 'Extension', 'DataResource', 'DataSet', 'Retrieval', 'Query', or 'Computation'.
doc.add_namespace('log', 'http://datamechanics.io/log/') # The event log.
doc.add_namespace('bdp', 'https://www.50states.com/bio/mass.htm')

this_script = doc.agent('alg:'+contributor+'#public_libraries', {prov.model.PROV_TYPE:prov.model.PROV['SoftwareAgent'], 'ont:Extension':'py'})
resource = doc.entity('bdp:wc8w-nujj', {'prov:label':'311, Service Requests', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
get_names = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
doc.wasAssociatedWith(get_names, this_script)
doc.usage(get_names, resource, startTime, None,
    {prov.model.PROV_TYPE:'ont:Retrieval',
    'ont:Computation':'Data cleaning'
    }
)

fp = doc.entity('dat:'+contributor+'#public_libraries', {prov.model.PROV_LABEL:'Public Libraries', prov.model.PROV_TYPE:'ont:DataSet'})
doc.wasAttributedTo(fp, this_script)
doc.wasGeneratedBy(fp, get_names, endTime)
doc.wasDerivedFrom(fp, resource, get_names, get_names, get_names)

repo.logout()
doc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Database.logout() is deprecated


<ProvDocument>

In [10]:
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

document
  prefix alg <http://datamechanics.io/algorithm/>
  prefix dat <http://datamechanics.io/data/>
  prefix ont <http://datamechanics.io/ontology#>
  prefix log <http://datamechanics.io/log/>
  prefix bdp <https://www.50states.com/bio/mass.htm>
  
  agent(alg:mmao95_Dongyihe_weijiang_zhukk#public_libraries, [prov:type='prov:SoftwareAgent', ont:Extension="py"])
  entity(bdp:wc8w-nujj, [prov:label="311, Service Requests", prov:type="ont:DataResource", ont:Extension="json"])
  activity(log:uuidcebab449-781d-47a0-9435-77998ca49aa5, -, -)
  wasAssociatedWith(log:uuidcebab449-781d-47a0-9435-77998ca49aa5, alg:mmao95_Dongyihe_weijiang_zhukk#public_libraries, -)
  used(log:uuidcebab449-781d-47a0-9435-77998ca49aa5, bdp:wc8w-nujj, -, [prov:type="ont:Retrieval", ont:Computation="Data cleaning"])
  entity(dat:mmao95_Dongyihe_weijiang_zhukk#public_libraries, [prov:label="Public Libraries", prov:type="ont:DataSet"])
  wasAttributedTo(dat:mmao95_Dongyihe_weijiang_zhukk#public_libraries, alg:mmao9